# Uploading Batch Input File
 - batch file 보낼 때 너무 많이 보내지 말고 구분해서 보내기

In [2]:
from getpass import getpass

openai_api_key = getpass("OPENAI_API_KEY")

In [3]:
from openai import OpenAI
import json
import pandas as pd

client = OpenAI(api_key=openai_api_key)

In [247]:
batch_input_file = client.files.create(
    file=open("VocaList_batch_5.jsonl","rb"),
    purpose="batch"
)

print(batch_input_file)

FileObject(id='file-6dt3Vr8ob7VpSdaJG8f4Fo', bytes=1859539, created_at=1739946925, filename='VocaList_batch_5.jsonl', object='file', purpose='batch', status='processed', status_details=None)


# Create the Batch

In [248]:
# 업로드한 배치 입력 파일 ID 저장
batch_input_file_id = batch_input_file.id
print(batch_input_file_id)

file-6dt3Vr8ob7VpSdaJG8f4Fo


In [249]:
batch_info = client.batches.create(
    input_file_id = batch_input_file_id,
    endpoint = "/v1/chat/completions", # 챗봇 api
    completion_window = "24h", # 현재는 이 기능만 가능
    metadata = {
        "description": "Mid & High example sentence 5"
    }
)

In [250]:
response_json = json.loads(batch_info.json())
response_output = json.dumps(response_json, ensure_ascii=False, indent = 2)
print(response_output)

{
  "id": "batch_67b57bb55d688190b6155f46575a8915",
  "completion_window": "24h",
  "created_at": 1739946933,
  "endpoint": "/v1/chat/completions",
  "input_file_id": "file-6dt3Vr8ob7VpSdaJG8f4Fo",
  "object": "batch",
  "status": "validating",
  "cancelled_at": null,
  "cancelling_at": null,
  "completed_at": null,
  "error_file_id": null,
  "errors": null,
  "expired_at": null,
  "expires_at": 1740033333,
  "failed_at": null,
  "finalizing_at": null,
  "in_progress_at": null,
  "metadata": {
    "description": "Mid & High example sentence 5"
  },
  "output_file_id": null,
  "request_counts": {
    "completed": 0,
    "failed": 0,
    "total": 0
  }
}


# Checking the Status of Batch
 - validating : the input file is being validated before the batch can begin
 - failed : the input file has failed the validation process
 - in_progress : the input file was successfully validated and the batch is currently being run
 - finalizing : the batch has completed and the results are being prepared
 - completed : the batch has been completed and the results are ready
 - expired : the batch was not able to be completed within the 24-hour time window
 - cancelling : the batch is being cancelled (may take up to 10 minutes)
 - cancelled : the batch was cancelled

*split data*
- batch_id = 'batch_67b462a042608190a541290b8094701e' : split1_voca_structure : completed
- batch_id = 'batch_67b568c1eab88190a5b1f58fc7e99e6c' : split2_voca_structure : completed
- batch_id = 'batch_67b56915b4208190969275dc62a1791a' : split3_voca_structure : completed
- batch_id = 'batch_67b462df195081909305d512936b1c14' : split4_voca_structure : completed
- batch_id = 'batch_67b569320c208190bec53dcbfdc7ea20' : split5_voca_structure : completed
- batch_id = 'batch_67b4630a90748190a56fcb216c5fe468' : split6_voca_structure : completed
- batch_id = 'batch_67b462860bcc8190b3074b9a01cf48fe' : split7_voca_structure : completed

에러 부분
batch5 - batch_67b57bb55d688190b6155f46575a8915

In [259]:
batch_id = "batch_67b57bb55d688190b6155f46575a8915"
batch = client.batches.retrieve(batch_id)
batch_json = json.loads(batch.json())
batch_json

{'id': 'batch_67b57bb55d688190b6155f46575a8915',
 'completion_window': '24h',
 'created_at': 1739946933,
 'endpoint': '/v1/chat/completions',
 'input_file_id': 'file-6dt3Vr8ob7VpSdaJG8f4Fo',
 'object': 'batch',
 'status': 'in_progress',
 'cancelled_at': None,
 'cancelling_at': None,
 'completed_at': None,
 'error_file_id': None,
 'errors': None,
 'expired_at': None,
 'expires_at': 1740033333,
 'failed_at': None,
 'finalizing_at': None,
 'in_progress_at': 1739946934,
 'metadata': {'description': 'Mid & High example sentence 5'},
 'output_file_id': None,
 'request_counts': {'completed': 991, 'failed': 0, 'total': 1000}}

## 시간 확인

In [201]:
from datetime import datetime
import pytz

kst = pytz.timezone('Asia/Seoul')

# 변환 함수
def convert_timestamp(starttime, endtime):
    st = datetime.utcfromtimestamp(starttime)
    end = datetime.utcfromtimestamp(endtime)
    dt_kst_st = st.replace(tzinfo=pytz.utc).astimezone(kst)
    dt_kst_end = end.replace(tzinfo=pytz.utc).astimezone(kst)

    return f"시작 시간 : {dt_kst_st}, 끝나는 시간 : {dt_kst_end}"

convert_timestamp(starttime = batch_json['created_at'], endtime = batch_json['completed_at'])


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_19724\1843218754.py:8: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  st = datetime.utcfromtimestamp(starttime)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_19724\1843218754.py:9: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  end = datetime.utcfromtimestamp(endtime)


'시작 시간 : 2025-02-18 19:38:02+09:00, 끝나는 시간 : 2025-02-19 11:51:53+09:00'

# Retrieving the Results
 - file-EVCbFfNykgV9HphK2h9wsk

In [244]:
file_response = client.files.content('file-W689xmYKwV6dNNq4irY8BA') # output_file_id 이용
print(file_response.text)

{"id": "batch_req_67b56c2405dc81908c70666ca6e9cbd5", "custom_id": "request-1", "response": {"status_code": 200, "request_id": "ba3f6b3b37ae4d97fb83870143827d4a", "body": {"id": "chatcmpl-B2WmGlyw1dWuwHneEGO6bpUWfUgd4", "object": "chat.completion", "created": 1739942352, "model": "o3-mini-2025-01-31", "choices": [{"index": 0, "message": {"role": "assistant", "content": "{\n  \"example_sentence\": \"We use tests to determine the correct answer.\",\n  \"example_sentence_kor\": \"\uc6b0\ub9ac\ub294 \ud14c\uc2a4\ud2b8\ub97c \uc0ac\uc6a9\ud558\uc5ec \uc62c\ubc14\ub978 \ub2f5\uc744 \ubc1d\ud600\ub0b4\uc694.\"\n}", "refusal": null}, "finish_reason": "stop"}], "usage": {"prompt_tokens": 347, "completion_tokens": 691, "total_tokens": 1038, "prompt_tokens_details": {"cached_tokens": 0, "audio_tokens": 0}, "completion_tokens_details": {"reasoning_tokens": 640, "audio_tokens": 0, "accepted_prediction_tokens": 0, "rejected_prediction_tokens": 0}}, "service_tier": "default", "system_fingerprint": "fp

In [245]:
list = file_response.text.split("\n")
list = [item for item in list if item.strip()]
list
len(list)


1000

In [246]:
import json

sample_output = []

# sample_list가 JSON 문자열 리스트라고 가정
for sample in list:
    sample_json = json.loads(sample)  # 문자열을 JSON 딕셔너리로 변환
    content = sample_json["response"]["body"]["choices"][0]["message"]["content"]
    content_json = json.loads(content)  # 내부 JSON 문자열도 변환

    sample_output.append(f"{sample_json["custom_id"]} 번째")
    sample_output.append(content_json)

    print(json.dumps(content_json, indent=2, ensure_ascii=False))  # 한글 출력 가능
    break


{
  "example_sentence": "We use tests to determine the correct answer.",
  "example_sentence_kor": "우리는 테스트를 사용하여 올바른 답을 밝혀내요."
}


# batch 결과

In [232]:
for sample in list:
    sample_json = json.loads(sample)
    content = sample_json["response"]["body"]["choices"][0]["message"]["content"]
    content_json = json.loads(content)
    content_json['example_sentence'] = content_json['example_sentence'].replace("\t"," ")
    content_json['example_sentence_kor'] = content_json['example_sentence_kor'].replace("\t"," ")
    print(json.dumps(content_json, indent=2, ensure_ascii=False))
    break

{
  "example_sentence": "I always say hello when I read signs.",
  "example_sentence_kor": "나는 항상 표지판을 읽을 때 인사해요."
}


# CSV 파일로 변환

In [233]:
def JsonToCSV(data:json, file_name:str) -> json:
    data_list = []

    for sample in data:
        # json 변환환
        sample_json = json.loads(sample)
        content = sample_json["response"]["body"]["choices"][0]["message"]["content"]
        content_json = json.loads(content)
        content_json['example_sentence'] = content_json['example_sentence'].replace("\t"," ")
        content_json['example_sentence_kor'] = content_json['example_sentence_kor'].replace("\t"," ")

        # 학습 목표
        example_sentence = content_json["example_sentence"]
        example_sentence_kor = content_json["example_sentence_kor"]

        # 데이터 만들기
        csv_data = {
            "example_english": example_sentence,
            "example_korean": example_sentence_kor,
        }

        data_list.append(csv_data)

    df = pd.DataFrame(data_list)

    # CSV 저장
    df.to_csv(f"{file_name}.csv", index=False, encoding='utf-8-sig')

    print(f"CSV 변환 완료 파일명: {file_name}.csv")


In [234]:
name = 'ExampleSentence_voca_batch_7'
JsonToCSV(list, name)

CSV 변환 완료 파일명: ExampleSentence_voca_batch_7.csv


In [235]:
len(list)

1251

In [241]:
df1 = pd.read_csv('ExampleSentence_voca_batch_1.csv')
df2 = pd.read_csv('ExampleSentence_voca_batch_2.csv')
df3 = pd.read_csv('ExampleSentence_voca_batch_3.csv')
df4 = pd.read_csv('ExampleSentence_voca_batch_4.csv')
df5 = pd.read_csv('ExampleSentence_voca_batch_5.csv')
df6 = pd.read_csv('ExampleSentence_voca_batch_6.csv')
df7 = pd.read_csv('ExampleSentence_voca_batch_7.csv')

merged_data = pd.concat([df1, df2, df3, df4, df5, df6, df7], axis=0, ignore_index=True)
merged_data.shape
merged_data.to_csv("ExampleSentence_Voca_Mid(total).csv", encoding="utf-8-sig")

# Cancelling a Batch

In [237]:
# cancel_batch = client.batches.cancel("")
# batch_json = json.loads(cancel_batch.json())
# batch_ouput = json.dumps(batch_json, ensure_ascii=False, indent=2)
# print(batch_ouput)